# Plott sirkelbuer og avstander veg  - antatt posisjon kamera

Inpsirert av https://towardsdatascience.com/a-very-simple-demo-of-interactive-controls-on-jupyter-notebook-4429cf46aabd 

Klippet fra 
https://github.com/tirthajyoti/Interactive_Machine_Learning/blob/master/Curve_fit_widget_1.ipynb 

# Problemstilling: Vegbilder og "10 meter fram" 

Vegbildene tas fra en plattform med state-of-the-art navigasjonsplattform, med GNSS, trippteller og whatnot. Av ukjente grunner tar man data fra dette posisjoneringssystemet og ber om vegnettstilknytning 10meter **foran** bilen. Dette gir en del interessante geometriutfordringer som vi her skal utforske. 

I tillegg er dette en god problemstilling for interactive widgets. 

Parameter **Svingradius** er den reelle svingradiusen til målebilen. 

**TilSide** er avstanden fra målebil til midten av vegen, der vegnettet typisk befinner seg. 

Her viser vi hvordan geometrien blir i en venstresving. Målebilen kjører rakt fram fra venstre mot høyre, men ligger i en 

![eksempel](sirkelbueplott_fig1.png) 


In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
def finnavstand( Radius ): 
    h1 = -1*Radius + np.sqrt( 4 * Radius**2 + 400   ) / 2 
    h2 = -1*Radius - np.sqrt( 4 * Radius**2 + 400   ) / 2 
    h = max( [h1, h2])
    return h

def sirkelbue(Svingradius, TilSide ): 
    # Plotter et utsnitt av sirkelbue
    Gulstripeavstand = TilSide
    Offset = Gulstripeavstand
    Radius = Svingradius
    
    debug = False 
    
    # finner theta, vinkelbue for utkjørt distanse som tilsvarer vektor [0,10], men langs sirkelbuen
    theta0 = np.arctan( [ 10 / Radius ] )[0]  
    th0 = theta0 # + 1 * np.pi / 2 # Legger på for å plotte på annet sted langs sirkelbuen
    
    # Regner ut differansen 
    h = finnavstand( Radius )
    print(  'Avstand påstått kameraposisjon - bilens reelle posisjon=', str(round( h, 2)) + 'm') 

    
    theta = np.linspace( 3 * np.pi / 2, th0 + 3*  np.pi/2, 100) 
    x = Radius * np.cos( theta )
    y = Radius * np.sin( theta ) + Radius 

    plt.figure(figsize=(8,5))
    


    # Sirkelbue nr 2, som representerer senterlinje
    R2 = Radius - Gulstripeavstand
    x2 = R2 * np.cos( theta )
    y2 = R2 * np.sin( theta ) + Radius 
    plt.plot( x2, y2, '--', c='y', lw=10)

    # differanse reell sirkelbue - posisjonsgjetting
    if Gulstripeavstand > 0: 
        plt.plot( [x2[-1], 10], [y2[-1], 0], '--', c='r', lw=1)
    plt.plot( [x[-1], 10], [y[-1], 0], 'r-', lw=1)
    
    # Kjørt avstand langs sirkelbue 
    dR = Radius * theta0 
    dRVegnett = R2 * theta0  
    
    if Offset > 0: 
        print( 'Avstand påstått kameraposisjon - posisjon vegnett=', str( round( h+Offset, 2)) + 'm'  ) 
        print( 'Påstått distanse langs vegnettet =', str( round( dRVegnett, 2))+'m')
    
    # Målebilens kjøring
    plt.plot( x, y,c='k', lw=3 )
        
        
    # Legger på tekst 
    # plt.text( 0, -0.5, 'Målebil posisjon' ) # , ha='center')
    plt.annotate( 'Målebil startposisjon', xy=(0,0), xytext=(0,-1), arrowprops=dict(facecolor='black', width=0.5 ))
    
    plt.annotate( 'Vegnett startposisjon', xy=(0,Offset), xytext=(0,Offset+1), 
                 arrowprops=dict(facecolor='black', width=0.5 ))
    
    
    plt.text( 10.1, 0, 'Påstått kameraposisjon (10 m rett fram)')
    plt.text( x[-1]+0.2, y[-1], 'Påstått kameraposisjon tilsvarer ' + str( round( dR, 2)) + 'm langs sirkelbue')
    
    xtext = np.mean( [10, x[-1]]  )
    ytext = np.mean( [0, y[-1]]  )
    plt.text( xtext+0.5, ytext, 'Avstand påstått - sirkelbue: ' + str( round( h, 2)) + 'm', color='r')
    
    if Gulstripeavstand > 0: 
        plt.text( x2[-1]+0.2, y2[-1], 'Vegnett påstått posisjon, ' + str( round( dRVegnett, 2)) + 'm fra start' )
    
    # Full sirkel, for morro skyld og debugging
    if debug: 
        thetaFull = np.linspace( 0, 2*np.pi, 100)
        x4 = Radius * np.cos( thetaFull )
        y4 = Radius * np.sin( thetaFull ) + Radius
        plt.plot( x4, y4 )

    
    # Rak linje 10m rett fram = gjetting på posisjon
    plt.arrow( 0, 0, 10, 0, color='b', width=0.1, length_includes_head=True )        
        
        
    plt.axes().set_aspect('equal', 'datalim')
    # plt.box( False )
    plt.axis( 'Off')
    plt.show( )
#     return theta

In [3]:
y = interactive( sirkelbue, Svingradius=[10, 15, 20, 25, 30, 40, 50, 75, 100], TilSide=[0, 0.5, 1, 1.5, 2, 3, 4, 5] )
display( y) 

interactive(children=(Dropdown(description='Svingradius', options=(10, 15, 20, 25, 30, 40, 50, 75, 100), value…